# packages

In [109]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("pandas")
install("numpy")
install("matplotlib")

# imports

In [1]:
import pandas as pd
import numpy as np
import json
import csv
import os

# Read Extraction Data from CSV

In [2]:
df = pd.read_csv(filepath_or_buffer="data/04 extraction consolidation results_reduced.csv",sep=";")

In [3]:
df2 = pd.read_csv("data/04 extraction consolidation results_backup.csv", sep="|")

In [4]:
print(list(df2))

['Title;DT Definition;virtual space;Kritzinger;DT is specific;modeling language original;note on modeling language;model type original;modeling language;language fidelity category;language content category;language content category consolidated;model type;model processing technique;model usage technique;source;source category;source sub-category 1;source model target category;target;target category;target sub-category;target model target category;purpose of MDE application;purpose of DT;DTPurposeCategory;use case domain;system lifecycle phase;twinning target;twin lifecycle phase;purpose /expected benefit of using models;open challenges;Technological Readiness Level;notes;Use Case']


In [5]:
df2.drop(['source', 'source category', 'source model target category'])

KeyError: "['source', 'source category', 'source model target category'] not found in axis"

In [6]:
result = df['Title'].to_list()

In [7]:
result = list(dict.fromkeys(result))
print(len(result))

48


In [8]:
titles = list(df.columns)

In [9]:
metadata = pd.read_csv("./data/search_results.csv", sep=";", encoding="latin-1")

In [10]:
metadata = metadata.drop_duplicates(subset=['Title'])

In [11]:
metadata

,Title,Publication Title,Authors,Publication Year,Document Identifier,Publisher
0,Air Quality Management: An Exemplar for Model-...,2021 ACM/IEEE International Conference on Mode...,H. S. Govindasamy; R. Jayaraman; B. Taspinar; ...,2021.0,IEEE Conferences,IEEE
1,The Research of Aero-engine Digital Twin Model...,2021 IEEE 10th Data Driven Control and Learnin...,K. Liu; P. Wang; Q. Li,2021.0,IEEE Conferences,IEEE
2,Generative and Model-driven SDK development fo...,2021 26th IEEE International Conference on Eme...,N. Braunisch; M. Ristin-Kaufmann; R. Lehmann; ...,2021.0,IEEE Conferences,IEEE
3,Models Meet Data: Challenges to Create Virtual...,2021 ACM/IEEE International Conference on Mode...,M. Van Den Brand; L. Cleophas; R. Gunasekaran;...,2021.0,IEEE Conferences,IEEE
4,Towards a Reference Architecture for Leveragin...,2020 25th IEEE International Conference on Eme...,D. Lehner; S. Wolny; A. Mazak-Huemer; M. Wimmer,2020.0,IEEE Conferences,IEEE
...,...,...,...,...,...,...
276,UAV-Assisted Three-Dimensional Spectrum Mappin...,SYMMETRY-BASEL,"Du, XF; Zhu, QM; Ding, GR; Li, J; Wu, QH; Lan,...",NaN,J,NaN
278,Decentralized Autonomous Operations and Organi...,IEEE TRANSACTIONS ON SYSTEMS MAN CYBERNETICS-S...,"Zhao, C; Dai, XY; Lv, YS; Niu, JL; Lin, YL",NaN,J,NaN
279,A virtual model knowledge updating method driv...,INTERNATIONAL JOURNAL OF ADVANCED MANUFACTURIN...,"Luo, RP; Sheng, BY; Fu, GC; Huang, YZ; Jian, Y...",NaN,J,NaN
281,Data-Driven Methodology to Support Long-Lastin...,SUSTAINABILITY,"Gutierrez-Franco, E; Mejia-Argueta, C; Rabelo, L",NaN,J,NaN


In [12]:
data = df.merge(metadata, how='left', on='Title')
#unique function

In [13]:
data

,Title,model processing technique,source,source category,source sub-category,source model target category,target,target category,target category 2,target sub-category,...,system lifecycle phase,twinning target,twin lifecycle phase,Technological Readiness Level,Use Case,Publication Title,Authors,Publication Year,Document Identifier,Publisher
0,A pattern catalog for augmenting Digital Twin ...,Model to Model Transformation,behavior model,behavior model,state-based behavior,PT,structure + behavior model (DTML),behavior + structure model,NaN,state-based behavior,...,Design,Individual Systems,As-Designed,TRL 1-3,3D Printer,At-Automatisierungstechnik,Lehner D.; Sint S.; Eisenberg M.; Wimmer M.,2023.0,Article,De Gruyter Oldenbourg
1,A pattern catalog for augmenting Digital Twin ...,Model to Model Transformation,structure model,structural model,data model,PT,structure + behavior model (DTML),behavior + structure model,NaN,NaN,...,Design,Individual Systems,As-Designed,TRL 1-3,3D Printer,At-Automatisierungstechnik,Lehner D.; Sint S.; Eisenberg M.; Wimmer M.,2023.0,Article,De Gruyter Oldenbourg
2,Case Study on Automated and Continuous Reliabi...,NaN,SysML v2 action sequences,behavior model,state-based behavior,NaN,Markov chains,behavior model,NaN,state-based behavior,...,Operate,NaN,As-Operated,NaN,NaN,Proceedings of the 25th International Conferen...,"Grimmeisen P,Wortmann A,Morozov A",2022.0,Conference Paper,ACM
3,Case Study on Automated and Continuous Reliabi...,Model to Model Transformation,SysML v2 structure models,structural model,NaN,NaN,Fault Trees,behavior model,NaN,state-based behavior,...,Operate,Individual Systems,As-Operated,TRL 1-3,NaN,Proceedings of the 25th International Conferen...,"Grimmeisen P,Wortmann A,Morozov A",2022.0,Conference Paper,ACM
4,Combining Low-Code Programming and SDL-Based M...,NaN,SDL model,behavior model,state-based behavior,PT,Discrete Event System Specification (formal mo...,behavior model,NaN,continuous behavior,...,Design,Processes,As-Designed,TRL 4-6,NaN,2021 ACM/IEEE International Conference on Mode...,X. P. i. Palomés; P. Tuset-Peiró; P. F. i. Casas,2021.0,IEEE Conferences,IEEE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Knowledge driven rapid development of white bo...,Model to Model Transformation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,IECON 2021  47th Annual Conference of the IEE...,A. B.; S. R.C.; B. B.; R. Dhakshinamoorthy; S....,2021.0,IEEE Conferences,IEEE
102,Meta-model-based shop-floor digital twin archi...,Model Interpretation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Robotics and Computer-Integrated Manufacturing,Yang X.; Liu X.; Zhang H.; Fu L.; Yu Y.,2023.0,Article,Elsevier Ltd
103,The OMiLAB Digital Innovation environment: Agi...,Model to Model Transformation,BPMN,behavior model,state-based behavior,PT,NaN,NaN,NaN,NaN,...,Design,Individual Systems,As-Designed,TRL 4-6,NaN,Computers in Industry,Karagiannis D.; Buchmann R.A.; Utz W.,2022.0,Article,Elsevier B.V.
104,Towards a Digital Twin for Cyber-Physical Prod...,Model to Model Transformation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Proceedings of the 23rd ACM/IEEE International...,"Niati A,Selma C,Tamzalit D,Bruneliere H,Mebark...",2020.0,Conference Paper,ACM


In [14]:
del metadata
del df
del titles

In [15]:
index = 0
print('{', end="")
for row in data:
    print(index, row)
    if index == 0:
        #print('\''+row+'\'', end="")
        index += 1
        continue
    #print(', '+'\''+row+'\'', end="")
    index += 1
print('}')

{0 Title
1 model processing technique
2 source
3 source category
4 source sub-category
5 source model target category
6 target
7 target category
8 target category 2
9 target sub-category
10 target model target category
11 use case domain
12 system lifecycle phase
13 twinning target
14 twin lifecycle phase
15 Technological Readiness Level
16 Use Case
17 Publication Title
18 Authors
19 Publication Year
20 Document Identifier
21 Publisher
}


# create JSON from CSV

In [16]:
arrayindexes2 = [10,12,13,14,15,16,17,18,19,20]

In [17]:
model_trafos = 'modelTrafos'
modelling_language = 'modeling language'

In [18]:
keys = ['Title', 'DT Definition', 'virtual space', 'Kritzinger', 'DT is specific', 'modeling language original', 'note on modeling language', 'model type original', 'modeling language', 'language fidelity category', 'language content category', 'language content category consolidated', 'model type', 'model processing technique', 'model usage technique', 'source', 'source category', 'source sub-category 1', 'source model target category', 'target', 'target category', 'target sub-category', 'target model target category', 'purpose of MDE application', 'purpose of DT', 'DTPurposeCategory', 'use case domain', 'system lifecycle phase', 'twinning target', 'twin lifecycle phase', 'purpose /expected benefit of using models', 'open challenges', 'Technological Readiness Level', 'notes', 'Use Case', 'Publication Title', 'Authors', 'Publication Year', 'Document Identifier', 'Publisher']

In [19]:
keys2 = ['Title', 'model processing technique', 'source', 'source category', 'source sub-category', 'source model target category', 'target', 'target category', 'target category 2', 'target sub-category', 'target model target category', 'use case domain', 'system lifecycle phase', 'twinning target', 'twin lifecycle phase', 'Technological Readiness Level', 'Use Case', 'Publication Title', 'Authors', 'Publication Year', 'Document Identifier', 'Publisher']

In [20]:
out_menge_2 = {}

In [21]:
print(out_menge_2)

{}


In [22]:
for key in result:
    out_menge_2[key] = {**{modelling_language:{}, model_trafos:{}}, **{keys2[x]: None for x in arrayindexes2}}

In [23]:
data.reset_index()
for title in result:
    rownum = 0
    
    for index, row in data.iterrows():
        if title == row['Title'] and not pd.isna(title):
            rowcopy = row.__deepcopy__()

            if row['use case domain'] is not None and not pd.isna(row['use case domain']):
                out_menge_2[title]['use case domain'] = row['use case domain']
            
            if row['twinning target'] is not None and not pd.isna(row['twinning target']):
                if out_menge_2[title]['twinning target'] is None:
                    out_menge_2[title]['twinning target'] = row['twinning target']
                else:
                    out_menge_2[title]['twinning target'] += ";"
                    out_menge_2[title]['twinning target'] +=  row['twinning target']

            if row['Technological Readiness Level'] is not None and not pd.isna(row['Technological Readiness Level']):
                out_menge_2[title]['Technological Readiness Level'] = row['Technological Readiness Level']
            
            if row['Use Case'] is not None and not pd.isna(row['Use Case']):
                out_menge_2[title]['Use Case'] = row['Use Case']

            if row['Publication Title'] is not None and not pd.isna(row['Publication Title']):
                out_menge_2[title]['Publication Title'] = row['Publication Title']
            
            if row['Authors'] is not None and not pd.isna(row['Authors']):
                out_menge_2[title]['Authors'] = row['Authors']
            
            if row['Publication Year'] is not None and not pd.isna(row['Publication Year']):
                out_menge_2[title]['Publication Year'] = row['Publication Year']

            if row['Document Identifier'] is not None and not pd.isna(row['Document Identifier']):
                out_menge_2[title]['Document Identifier'] = row['Document Identifier']

            if row['Publisher'] is not None and not pd.isna(row['Publisher']):
                out_menge_2[title]['Publisher'] = row['Publisher']
                
            if row['twin lifecycle phase'] is not None and not pd.isna(row['twin lifecycle phase']):
                out_menge_2[title]['twin lifecycle phase'] = row['twin lifecycle phase']
            
            # multiple Trafos
            arrayindexes2.append(0)
            rowjson = rowcopy.drop(row.iloc[arrayindexes2].index).to_json()
            try:
                out_menge_2[title][model_trafos][rownum] = rowjson
            except Exception as ex:
                out_menge_2[title][model_trafos] = {}
                out_menge_2[title][model_trafos][rownum] = rowjson
            #out_menge_2[title]['data'][rownum].append(rowjson)
            rownum+=1

In [24]:
json_object = json.dumps(out_menge_2, indent = 4) 
try:
    os.mkdir('target/')
    os.mkdir('target/img/')
except OSError as error:
    print(error)
with open("target/json/data_consolidated_semistructured__4.json", "w") as outfile:
    outfile.write(json_object)

[WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'target/'


# old

In [25]:
data.reset_index()
for title in result:
    rownum = 0
    
    for index, row in data.iterrows():
        if title == row['Title'] and not pd.isna(title):
            rowcopy = row.__deepcopy__()
            
            if row['DT Definition'] is not None and not pd.isna(row['DT Definition']):
                out_menge_1[title]['DT Definition'] = row['DT Definition']
                
                if row['virtual space'] is not None and not pd.isna(row['virtual space']):
                    out_menge_1[title]['virtual space'] = row['virtual space']
            
                if row['Kritzinger'] is not None and not pd.isna(row['Kritzinger']):
                    out_menge_1[title]['Kritzinger'] = row['Kritzinger']
            
                if row['DT is specific'] is not None and not pd.isna(row['DT is specific']):
                    out_menge_1[title]['DT is specific'] = row['DT is specific']

            if row['purpose of DT'] is not None and not pd.isna(row['purpose of DT']):
                out_menge_1[title]['purpose of DT'] = row['purpose of DT']

            if row['DTPurposeCategory'] is not None and not pd.isna(row['DTPurposeCategory']):
                out_menge_1[title]['DTPurposeCategory'] = row['DTPurposeCategory']

            if row['use case domain'] is not None and not pd.isna(row['use case domain']):
                out_menge_1[title]['use case domain'] = row['use case domain']
            
            if row['twinning target'] is not None and not pd.isna(row['twinning target']):
                if out_menge_1[title]['twinning target'] is None:
                    out_menge_1[title]['twinning target'] = row['twinning target']
                else:
                    out_menge_1[title]['twinning target'] += ";"
                    out_menge_1[title]['twinning target'] +=  row['twinning target']

            if row['purpose /expected benefit of using models'] is not None and not pd.isna(row['purpose /expected benefit of using models']):
                out_menge_1[title]['purpose /expected benefit of using models'] = row['purpose /expected benefit of using models']

            if row['open challenges'] is not None and not pd.isna(row['open challenges']):
                out_menge_1[title]['open challenges'] = row['open challenges']

            if row['Technological Readiness Level'] is not None and not pd.isna(row['Technological Readiness Level']):
                out_menge_1[title]['Technological Readiness Level'] = row['Technological Readiness Level']
            
            if row['notes'] is not None and not pd.isna(row['notes']):
                out_menge_1[title]['notes'] = row['notes']

            if row['Use Case'] is not None and not pd.isna(row['Use Case']):
                out_menge_1[title]['Use Case'] = row['Use Case']

            if row['Publication Title'] is not None and not pd.isna(row['Publication Title']):
                out_menge_1[title]['Publication Title'] = row['Publication Title']
            
            if row['Authors'] is not None and not pd.isna(row['Authors']):
                out_menge_1[title]['Authors'] = row['Authors']
            
            if row['Publication Year'] is not None and not pd.isna(row['Publication Year']):
                out_menge_1[title]['Publication Year'] = row['Publication Year']

            if row['Document Identifier'] is not None and not pd.isna(row['Document Identifier']):
                out_menge_1[title]['Document Identifier'] = row['Document Identifier']

            if row['Publisher'] is not None and not pd.isna(row['Publisher']):
                out_menge_1[title]['Publisher'] = row['Publisher']
                
            if row['twin lifecycle phase'] is not None and not pd.isna(row['twin lifecycle phase']):
                out_menge_1[title]['twin lifecycle phase'] = row['twin lifecycle phase']
            
            if row['modeling language'] is not None and not pd.isna(row['modeling language']):
                if out_menge_1[title]['modeling language'] is None:
                    out_menge_1[title]['modeling language'] = {}
                out_menge_1[title]['modeling language'][row['modeling language']] = {'fidelity':row['language fidelity category'],'category': row['language content category'],'consolidated_category':row['language content category consolidated'],'model_type':row['model type']}#.append(row['modeling language'])

            # multiple Trafos
            arrayindexes = [0,1,2,3,4,23,24,26,28,29,30,31,32,33,34,35,36,37,38,39]
            rowjson = rowcopy.drop(row.iloc[arrayindexes].index).to_json()
            try:
                out_menge_1[title][model_trafos][rownum] = rowjson
            except Exception as ex:
                out_menge_1[title][model_trafos] = {}
                out_menge_1[title][model_trafos][rownum] = rowjson
            #out_menge_1[title]['data'][rownum].append(rowjson)
            rownum+=1


KeyError: 'DT Definition'

In [ ]:
out_menge_1 = {}
arrayindexes = [1,2,3,4,23,24,26,28,29,30,31,32,33,34,35,36,37,38,39]
for key in result:
    out_menge_1[key] = {**{modelling_language:{}, model_trafos:{}}, **{keys[x]: None for x in arrayindexes}}
    
    
x1 = {keys[1]:None,
                        keys[2]:None,
                        keys[3]:None,
                        keys[4]:None,
                        keys[15]:None,
                        keys[16]:None,
                        keys[17]:None,
                        keys[19]:None, 

                        model_trafos:{},
                        keys[22]:None,
                        keys[23]:None, 
                        keys[24]:None, 
                        keys[25]:None, 
                        modelling_language:{},
                        
                        }

print(out_menge_1)

In [ ]:
out_menge_1

{'A framework for Model-Driven Engineering of resilient software-controlled systems': {'modeling language': {'SysML': {'fidelity': nan,
    'category': 'ontology',
    'consolidated_category': 'logical or physical',
    'model_type': 'Structure'},
   'Knowledge base model': {'fidelity': nan,
    'category': 'behavior',
    'consolidated_category': 'structure or behavior',
    'model_type': '??'}},
  'modelTrafos': {0: '{"modeling language original":"SysML","note on modeling language":null,"model type original":"structure","modeling language":"SysML","language fidelity category":null,"language content category":"ontology","language content category consolidated":"logical or physical","model type":"Structure","model processing technique":"Model to Model Transformation","model usage technique":null,"source":"SysML: BDD","source category":"behavior model","source sub-category 1":"state-based","source model target category":null,"target":"Knowledge Base of software digital twins (Metamodel 

In [ ]:
del rowjson
del rowcopy
del result
del rownum

# export JSON to /json/data_consolidated_semistructured.json

In [ ]:
json_object = json.dumps(out_menge_1, indent = 4) 
try:
    os.mkdir('target/')
    os.mkdir('target/img/')
except OSError as error:
    print(error)
with open("target/json/data_consolidated_semistructured__3.json", "w") as outfile:
    outfile.write(json_object)

[WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'target/'
